In [1]:
%reload_kedro

                    INFO     Resolved project path as:                                              ]8;id=708088;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=937494;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py#164\164]8;;\
                             C:\Users\9509298u\Documents\GitHub\OC_Projects\ocp7-scoring-model-clou                
                             d.                                                                                    
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/03/24 08:57:01] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=994724;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=330343;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#65\65]8;;\

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=301281;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=986078;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///C:/Users/9509298u/Documents/GitHub/OC_Projects/ocp                           
                             7-scoring-model-cloud/mlruns'                                                         

[06/03/24 08:57:02] INFO     Kedro project ocp7_scoring_model_cloud                                 ]8;id=513129;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=358896;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=329230;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=55083;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py#135\135]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=401123;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=691332;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py#141\141]8;;\

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.dummy import DummyClassifier
import lightgbm as lgb

import matplotlib.pyplot as plt

import mlflow

# Métrique d'erreur business

On propose de découper le problème posé en deux questions qui peuvent être résolues séparément : 

1) la question de l'évaluation de la probabilité de défaut des clients,
2) la question de l'attribution d'un crédit sur la base de cette probabilité de défaut estimée.

L'évaluation de la probabilité de défaut des clients peut être réalisée à l'aide d'un algorithme de classification. Ensuite il nous faut trouver une règle de décision opérationnelle sur la base de cette probabilité.

Soit $P(D) = p$, la probabilité de défaut du client, $M$ le montant du crédit demandé et $i$ le taux d'intérêt appliqué par l'entreprise. On a $\Pi_D(i) = -M $ le profit réalisé en cas de défaut du client, et $\Pi_\bar{D}(i) = iM$ le profit réalisé en cas de remboursement du prêt. Le profit espéré de la firme en fonction de $p$ et $i$ peut donc s'écrire : $$E(\Pi|i,p) =(1-p)iM - piM $$

On constate que cette fonction est croissante de $i$, décroissante de $p$ et qu'elle s'annule lorsque $$p=\frac{i}{1+i}$$

Cette relation nous permet de définir un seuil $\bar{p}(i)$ sur la probabilité de défaut du client pour l'attribution d'un prêt à un taux d'intérêt $i$ donné. 

Par exemple, si le taux d'intérêt est de 10%, $i=0.1$, $\bar{p}(0.1) = 0.09$. On proposera l'octroi du prêt si la probabilité de défaut prédite par le modèle est inférieure à 0.09, soit 9%. 

On pourrait également renverser la relation et proposer d'appliquer un taux d'intérêt plus élevé pour prendre en compte le risque du client : $$\bar{i}(p) = \frac{p}{1-p}$$

In [3]:
train_df = catalog.load("full_df_train")
#test_df = catalog.load("full_df_test")

[06/03/24 08:59:06] INFO     Loading data from full_df_train (ParquetDataset)...                ]8;id=604772;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=645975;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

In [4]:
def split_data(df, train_size=0.8, test_size=0.2, random_state=42):
    features = [f for f in df.columns if f not in ["SK_ID_CURR", "TARGET"]]
    X = df[features] # Features
    y = df["TARGET"]  # Target variable
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

def train_model(X_train, y_train, model):
    model.fit(X_train, y_train)
    return model

def evaluate_model(X_test, y_test, model):
    y_pred = model.predict_proba(X_test)[:,1]
    roc_auc = roc_auc_score(y_test, y_pred)
    return roc_auc

In [5]:
def train_and_evaluate_model(X, y, model, model_name, params):
    if mlflow.active_run():
        mlflow.end_run()
    # Start an MLflow run
    with mlflow.start_run(run_name=model_name):
        # Fitting model
        model = model.fit(X, y)
        y_probs = model.predict_proba(X)
        y_scores = y_probs[:, 1]  # get the probability of class 1
        # Compute ROC-AUC
        print("Calculating ROC-AUC score...")
        roc_auc = roc_auc_score(y, y_scores)
        
        # Log parameters, metrics, and model
        print("Logging parameters, metrics, and model with MLflow...")
        mlflow.log_params(params)
        mlflow.log_metric("ROC_AUC", roc_auc)
        mlflow.sklearn.log_model(model, "model")

        # Plot ROC curve and save as an artifact
        print("Plotting ROC curve...")
        fpr, tpr, _ = roc_curve(y, y_scores)
        fig = plt.figure(figsize=(6,6))
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        roc_curve_path = "roc_curve.png"
        plt.savefig(roc_curve_path)
        plt.close(fig)
        print("Saving ROC curve as an artifact...")
        mlflow.log_artifact(roc_curve_path)

        print("Training and evaluation completed.")
        return roc_auc


In [6]:
mlflow.set_tracking_uri("mlruns")


In [29]:
mlflow.set_experiment("OCP7_modelisation_030624_1")

# Define your dataset
X_train, X_test, y_train, y_test = split_data(train_df.sample(frac=0.5))

# Dummy classifier

params_dummy = {'strategy': 'stratified'}
model_dummy = DummyClassifier(**params_dummy)
train_and_evaluate_model(X_train, y_train, model_dummy, "Dummy Classifier", params_dummy)

# Random Forest
params_rf = {'n_estimators': 50, 'max_depth': 10}
model_rf = RandomForestClassifier(**params_rf)
train_and_evaluate_model(X_train, y_train, model_rf, "Random Forest", params_rf)

# Logistic Regression
params_lr = {'C': 0.1}
model_lr = LogisticRegression(**params_lr)
train_and_evaluate_model(X_train, y_train, model_lr, "Logistic Regression", params_lr)

# LightGBM
params_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

model_lgb = lgb.LGBMClassifier(**params_lgb)
train_and_evaluate_model(X=X_train, y=y_train, model=model_lgb, model_name="LightGBM", params=params_lgb)

Calculating ROC-AUC score...
Logging parameters, metrics, and model with MLflow...
Plotting ROC curve...
Saving ROC curve as an artifact...
Training and evaluation completed.
Calculating ROC-AUC score...
Logging parameters, metrics, and model with MLflow...
Plotting ROC curve...
Saving ROC curve as an artifact...
Training and evaluation completed.
Calculating ROC-AUC score...
Logging parameters, metrics, and model with MLflow...
Plotting ROC curve...
Saving ROC curve as an artifact...
Training and evaluation completed.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[L

0.8242152620270637

On trouve que le modèle LightGBM est le plus performant. On va donc essayer de l'améliorer en utilisant une validation croisée.

In [7]:
# LightGBM
mlflow.set_experiment("OCP7_modelisation_030624_1")

# Define your dataset
X_train, X_test, y_train, y_test = split_data(train_df.sample(frac=0.5))
params_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

model_lgb = lgb.LGBMClassifier(**params_lgb)
train_and_evaluate_model(X=X_train, y=y_train, model=model_lgb, model_name="LightGBM", params=params_lgb)

2024/06/03 09:00:45 INFO mlflow.tracking.fluent: Experiment with name 'OCP7_modelisation_030624_1' does not exist. Creating a new experiment.


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Number of positive: 9967, number of negative: 113036
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.097384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 44426
[LightGBM] [Info] Number of data p

0.823514893816781

In [7]:
# Create a training and testing dataset
X_train, X_test, y_train, y_test = split_data(train_df, train_size=0.3, test_size=0.7)

In [10]:
params_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'n_estimators':10000,
    'verbose': -1
}

param_grid = {
    'num_leaves': [34, 65], 
    'learning_rate': [0.02, 0.05],  
    'max_depth': [8, 12]
}

model_lgb = lgb.LGBMClassifier(**params_lgb)

In [17]:
grid_search = GridSearchCV(estimator=model_lgb, param_grid=param_grid, cv=2, scoring='roc_auc', verbose=10)

In [18]:
grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2; 1/8] START learning_rate=0.02, max_depth=8, num_leaves=34..............
[CV 1/2; 1/8] END learning_rate=0.02, max_depth=8, num_leaves=34;, score=0.733 total time= 1.5min
[CV 2/2; 1/8] START learning_rate=0.02, max_depth=8, num_leaves=34..............
[CV 2/2; 1/8] END learning_rate=0.02, max_depth=8, num_leaves=34;, score=0.734 total time= 1.5min
[CV 1/2; 2/8] START learning_rate=0.02, max_depth=8, num_leaves=65..............
[CV 1/2; 2/8] END learning_rate=0.02, max_depth=8, num_leaves=65;, score=0.726 total time= 2.2min
[CV 2/2; 2/8] START learning_rate=0.02, max_depth=8, num_leaves=65..............
[CV 2/2; 2/8] END learning_rate=0.02, max_depth=8, num_leaves=65;, score=0.730 total time= 2.1min
[CV 1/2; 3/8] START learning_rate=0.02, max_depth=12, num_leaves=34.............
[CV 1/2; 3/8] END learning_rate=0.02, max_depth=12, num_leaves=34;, score=0.739 total time= 1.4min
[CV 2/2; 3/8] START learning_rate=0.02, max_

GridSearchCV(cv=2,
             estimator=LGBMClassifier(metric='auc', n_estimators=10000,
                                      objective='binary', verbose=-1),
             param_grid={'learning_rate': [0.02, 0.05], 'max_depth': [8, 12],
                         'num_leaves': [34, 65]},
             scoring='roc_auc', verbose=10)

In [56]:
X_train, X_test, y_train, y_test = split_data(train_df)
train_set = lgb.Dataset(X_train, y_train)
val_set = lgb.Dataset(X_test, y_test)

In [61]:
# Get default hyperparameters
model = lgb.LGBMClassifier()
default_params = model.get_params()

In [62]:
# Cross validation with early stopping
cv_results = lgb.cv(default_params, train_set, metrics='auc', 
                    nfold=5, seed=42, callbacks=[lgb.early_stopping(stopping_rounds=100)])

[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.146458 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45280
[LightGBM] [Info] Number of data points in the train set: 196804, number of used features: 456
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.167373 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45280
[LightGBM] [Info] Number of data points in the train set: 196804, number of used features: 456
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unk

In [65]:
cv_results


{
    'valid auc-mean': [
        0.7030554191648362,
        0.7103788606383553,
        0.7139725044585108,
        0.7170087269043448,
        0.7202398504680867,
        0.7226871865193297,
        0.7242932003846297,
        0.7263610369208934,
        0.7281280035664908,
        0.7302422187738682,
        0.7319578035366889,
        0.7339613560261296,
        0.7355943872884947,
        0.7374468396920524,
        0.7388030922900846,
        0.740776291332296,
        0.7422195243827643,
        0.7439724479172949,
        0.745209770740302,
        0.7470515605987046,
        0.7482211934467908,
        0.7494348413166397,
        0.7511464164749384,
        0.7520760963390046,
        0.7528827264898352,
        0.7535505728172579,
        0.7541444707255797,
        0.7548721275380622,
        0.7554799719981391,
        0.7560092189738187,
        0.756497205848081,
        0.7570644279348142,
        0.757670804185479,
        0.7580927723415668,
        0.758549168326478

In [66]:
print('The maximum validation ROC AUC was: {:.5f} with a standard deviation of {:.5f}.'.format(cv_results['valid auc-mean'][-1], cv_results['valid auc-stdv'][-1]))
print('The optimal number of boosting rounds (estimators) was {}.'.format(len(cv_results['valid auc-mean'])))

The maximum validation ROC AUC was: 0.76980 with a standard deviation of 0.00325.
The optimal number of boosting rounds (estimators) was 100.


In [67]:
# Optimal number of esimators found in cv
model.n_estimators = len(cv_results['valid auc-mean'])

# Train and make predicions with model
model.fit(X_train, y_train)
preds = model.predict_proba(X_test)[:, 1]
baseline_auc = roc_auc_score(y_test, preds)

print('The baseline model scores {:.5f} ROC AUC on the test set.'.format(baseline_auc))

[LightGBM] [Info] Number of positive: 19804, number of negative: 226201
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.178898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45280
[LightGBM] [Info] Number of data points in the train set: 246005, number of used features: 456
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080502 -> initscore=-2.435540
[LightGBM] [Info] Start training from score -2.435540
The baseline model scores 0.77386 ROC AUC on the test set.


In [45]:
# test_df = catalog.load("full_df_test")
# pd.DataFrame({"SK_ID_CURR": test_df["SK_ID_CURR"], "TARGET": y_pred_test}).to_csv("../data/07_model_output/kaggle_output_df_200524.csv", index=False)